In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##### This data is from Kaggle and provides information about 500 applications for various universities and what their chance of admittance is. We will build a regression model from this to predict any future admission and chance of acceptence

#### Load in dataset

In [2]:
df = pd.read_csv('data/admissions_data.csv')
print(df.columns)

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')


#### Create feautures and labels

In [3]:
features = df.iloc[:,:-1]
labels = df.iloc[:,-1]
features = features.drop('Serial No.', axis=1)


In [4]:
print(features)
print(labels)

     GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research
0          337          118                  4  4.5   4.5  9.65         1
1          324          107                  4  4.0   4.5  8.87         1
2          316          104                  3  3.0   3.5  8.00         1
3          322          110                  3  3.5   2.5  8.67         1
4          314          103                  2  2.0   3.0  8.21         0
..         ...          ...                ...  ...   ...   ...       ...
495        332          108                  5  4.5   4.0  9.02         1
496        337          117                  5  5.0   5.0  9.87         1
497        330          120                  5  4.5   5.0  9.56         1
498        312          103                  4  4.0   5.0  8.43         0
499        327          113                  4  4.5   4.5  9.04         0

[500 rows x 7 columns]
0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496 

#### Split into training and test set

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
features_training_set, features_test_set, lables_training_set, labels_test_set = train_test_split(features, labels, 
                                                                                   test_size=0.20, random_state=42)

#### Scaling and Normalizing numerical data

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer


In [8]:
scaler = ColumnTransformer([('scale', StandardScaler(), ['GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research'])], remainder='passthrough')
#fit the scaler to the training data and test data
features_training_set_scale = scaler.fit_transform(features_training_set) 
features_test_set_scale = scaler.transform(features_test_set) 

#ColumnTransformer returns numpy arrays. Convert the features to dataframes
features_training_set_scale = pd.DataFrame(features_training_set_scale, columns = features_training_set.columns)
features_test_set_scale = pd.DataFrame(features_test_set_scale, columns = features_test_set.columns)

#### Creating a function of tensorflow model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
def regression_model():
    model = Sequential()
    input = InputLayer(input_shape = (features.shape[7], ))
    my_model.add(input)
    my_model.add(Dense(64, activation='relu'))
    my_model.add(Dense(1))
    print(my_model.summary())
    model.add()